In [34]:
import tensorflow as tf
import pandas as pd
import nltk, re, time
from langdetect import detect
from contractions import get_contractions
from sqlalchemy import create_engine
from pprint import pprint
from nltk.corpus import stopwords
import chars2vec
import sklearn.decomposition
import matplotlib.pyplot as plt
import itertools
import string
import re
from keras.models import model_from_json
from keras.models import load_model
import gensim
import os
import numpy as np
from sklearn.utils import resample
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import roc_auc_score
import tqdm
from keras.initializers import Constant
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


In [6]:
data = pd.read_csv('/Users/admin/Documents/Queens_Masters_Courses/Deep_Learning/course_project/best/pre_processed_steam_reviews_final2.csv')


In [14]:
data['content'].values.tolist()[1]

'game made lose hope played game open beta came na eu excited beacuse jake song former developer lineage game director game much fun pvp system update came resisted playing game quitted around came back end good erenor update came erenor update destroyed game even tho dead update came erenor update litterally destroyed might ask well grind new levels normal think player trying kill faction players players everyone quitted wait update included new armor believe needed obsidian ayanand gear costed reccomend garbageage play lineage prelude interlude private server try black desert online fan open world pvp'

In [18]:
print(data.head(5)['content'])

0    subjectively compilation mediocre minigames gr...
1    game made lose hope played game open beta came...
2    extremely visually pleasing enticing story sol...
3    battlefield sensations back game go back sourc...
4    games reason steam requirement retail copy goo...
Name: content, dtype: object


In [19]:
steam_reviews = [word_tokenize(word) for word in data['content'].values.tolist()]
print(steam_reviews[0:5])


[['subjectively', 'compilation', 'mediocre', 'minigames', 'group', 'friends', 'play', 'avoid'], ['game', 'made', 'lose', 'hope', 'played', 'game', 'open', 'beta', 'came', 'na', 'eu', 'excited', 'beacuse', 'jake', 'song', 'former', 'developer', 'lineage', 'game', 'director', 'game', 'much', 'fun', 'pvp', 'system', 'update', 'came', 'resisted', 'playing', 'game', 'quitted', 'around', 'came', 'back', 'end', 'good', 'erenor', 'update', 'came', 'erenor', 'update', 'destroyed', 'game', 'even', 'tho', 'dead', 'update', 'came', 'erenor', 'update', 'litterally', 'destroyed', 'might', 'ask', 'well', 'grind', 'new', 'levels', 'normal', 'think', 'player', 'trying', 'kill', 'faction', 'players', 'players', 'everyone', 'quitted', 'wait', 'update', 'included', 'new', 'armor', 'believe', 'needed', 'obsidian', 'ayanand', 'gear', 'costed', 'reccomend', 'garbageage', 'play', 'lineage', 'prelude', 'interlude', 'private', 'server', 'try', 'black', 'desert', 'online', 'fan', 'open', 'world', 'pvp'], ['extre

In [53]:
# Train Doc2vec model
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(steam_reviews)]
model = Doc2Vec(documents, vector_size = 300, negative = 5, min_count = 1, workers = 6)


In [66]:
# Test Doc2vec model
vector = model.infer_vector(["positive", "response"])
print(len(vector))
model.wv.most_similar('shit', topn = 10)


300


[('fucking', 0.9278612732887268),
 ('ass', 0.9196205139160156),
 ('fuck', 0.9130381941795349),
 ('trash', 0.8737033605575562),
 ('garbage', 0.8705238103866577),
 ('crap', 0.8675743937492371),
 ('sfucking', 0.8616911768913269),
 ('holy', 0.8486007452011108),
 ('conned', 0.8462146520614624),
 ('peice', 0.8435782194137573)]

In [65]:
def create_model(rev_rows, rev_cols):
    print("Building model...")
    model = Sequential()

#     embedding_layer = Embedding(num_words,
#                                EMBEDDING_DIM,
#                                embeddings_initializer = Constant(embedding_matrix),
#                                input_length = MAX_LENGTH,
#                                trainable = False)

#     model.add(embedding_layer)
    model.add(Dense(128, activation = 'relu', input_shape=(rev_rows, rev_cols)))
    # Currently 64, 0.4, 0.4
    model.add(GRU(units = 128, dropout = 0.4, recurrent_dropout = 0.4))
    model.add(Dense(1, activation = 'sigmoid'))

    adam = optimizers.Adam(lr = 0.0001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])

#     print('Summary of the built model...')
    print(model.summary())
    return model;


In [69]:
data['tok_content'] = data['content'].map(word_tokenize)


In [71]:
data['doc_vec'] = data['tok_content'].map(model.infer_vector)


In [161]:
doc_vec = data['doc_vec']
hours_played = data['hours_all']
comp = data['compensation']


In [162]:
from sklearn.preprocessing import MinMaxScaler
hours_played_scaled = minmax_scale(hours_played, feature_range= (0,1), axis = 0, copy = True)


In [147]:
# new_data = np.vstack((doc_vec, hours_played_scaled, comp))
# new_data = np.vstack((doc_vec, hours_played_scaled, comp])).reshape(46169,3)
doc_vec = doc_vec.reshape(1, 46169)
hours_played_scaled = hours_played_scaled.reshape(1, 46169)
comp = comp.values.reshape(1,46169)

# print(new_data.shape)
# print(new_data[0][2])

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [186]:
new_data = pd.concat([pd.Series(doc_vec), pd.Series(hours_played_scaled), pd.Series(comp)])
# new_data = new_data.reshape(46169, 3)
new_data_mat = new_data.as_matrix
print(new_data_mat)


<bound method NDFrame.as_matrix of 0        [0.005809192, 0.0013914531, -0.011887878, -0.0...
1        [-0.040256817, 0.004481264, 0.044122715, -0.05...
2        [0.012847292, -0.0012835631, -0.004706556, -0....
3        [0.0026366932, 0.014811996, -0.009830314, -0.0...
4        [-0.034745686, -0.038029794, -0.027089985, -0....
5        [0.010525068, 0.004605245, -0.007871234, 0.000...
6        [0.0057089925, 0.015486509, -0.011018858, 0.00...
7        [-0.00048519342, 0.0014340155, -0.0027292995, ...
8        [0.01546066, -0.00066498364, -0.021535309, -0....
9        [0.06429689, -0.07966819, -0.047984246, -0.073...
10       [-0.018333472, 0.0010086348, 0.014274573, 0.00...
11       [-0.0046486226, 0.014563507, -0.003147583, -0....
12       [-0.004585385, -0.0011561584, -0.016749717, -0...
13       [0.0065817055, 0.040071476, 0.02215917, 0.0311...
14       [-0.02656216, -0.0049655843, 0.03597522, 0.006...
15       [0.009882016, -0.0020508866, -0.010101916, -0....
16       [0.015375565

In [ ]:
def train_model(model, X, y):
    # Get class weights and early stopping obj
    class_weights = class_weight.compute_class_weight('balanced',
                                                np.unique(y_train),
                                                y_train)
    
    class_weight_dict = dict(enumerate(class_weights))
    print(class_weight_dict)
    
    callbacks = [EarlyStopping(monitor = 'val_loss', patience = 10),
            ModelCheckpoint(filepath = 'cv_model_checkpoint.h5', monitor = 'val_loss', save_best_only = True)]
    
    print("Training model...")
    result = model.fit(X, 
              y, 
              batch_size = 128,
              epochs = 25, 
              validation_data = (X_val_pad, y_val), 
              verbose = 2, 
              callbacks = callbacks, 
              class_weight = class_weight_dict, 
              shuffle = True)
    
    return result;


In [ ]:
create_model(128, 302)